# Fine Tuning LLM using Unsloth

In [2]:
import os
import json
import torch

print(f"torch Version: {torch.__version__}")
print(f"CUDA Available: {torch.cuda.is_available()}")

torch Version: 2.8.0+cu126
CUDA Available: True


In [3]:
dataset=[
    {
        "instruction": "Translate English to Tamil",
        "input":"Good morning!",
        "output":"Bonjour!"
    },
    {
        "instruction": "What is the capital of France?",
        "input":"Deep learning enables computers to learn complex patterns.",
        "output":"The capital of France is Paris"
    }
]

In [4]:
os.makedirs("data",exist_ok=True)
with open("data/dataset.json","w") as f:
    for ex in dataset:
      f.write(json.dumps(ex)+"\n")

print("Sample dataset saved.")

Sample dataset saved.


In [7]:
# install unsloth
!pip install unsloth[colab-new] torch trl peft transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.3/52.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 544.8/544.8 kB 11.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 28.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 9.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 131.7/131.7 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.9/312.9 kB 23.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 18.1 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


# Using Unsloth for fine-tune

In [8]:
from unsloth import FastLanguageModel
from transformers import TrainingArguments, Trainer,DataCollatorForLanguageModeling
from datasets import load_dataset

/tmp/ipython-input-1164755917.py:1: UserWarning: WARNING: Unsloth should be imported before transformers to ensure all optimizations are applied. Your code may run slower or encounter memory issues without these optimizations.

Please restructure your imports with 'import unsloth' at the top of your file.
  from unsloth import FastLanguageModel


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!


In [17]:
!pip install -U bitsandbytes

In [21]:
model_name ="Llama 3.2/1 B"

model = FastLanguageModel.from_pretrained(
    model_name=model_name,
      load_in_4bit=True,
  )
# Peft is the method to make architecture level changes
model = FastLanguageModel.get_peft_model(
    model,
    r=8,
    target_modules=["q_proj","v_proj","k_proj"],
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    use_gradient_checkpointing=True,
    )

FileNotFoundError: Llama 3.2/1 B/*.json (invalid repository id)

In [19]:
from math import trunc
dataset = load_dataset("json",data_files="data/dataset.json",split="train")

tokenizer = FastLanguageModel.get_tokenizer(
    model_name ="TinyLlama/TinyLlama-1.18-Chat-v1.0",
    max_seq_length = 512,
)

def tokenize_fn(example):
  prompt=f"### Instruction:\n{example['instruction']}\n\n### Input:\n{example['input']}\n\n### Response:\n{example['output']}"
  tokenized = tokenizer(
      prompt,
      truncation=True,
      max_length=512,
  )
  tokenized["labels"] = tokenized["input_ids"].copy()
  return tokenized

tokenized_dataset=dataset.map(tokenize_fn)

NameError: name 'model' is not defined

In [ ]:
training_args = TrainingArguments(
    output_dir ="finetuned_model",
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    learning_rate = 2e-4,
    logging_steps=1,
    num_train_epochs=1,
    bf16 = torch.cuda.is_bf16_supported(),
    fp16 = not torch.cuda.is_bf16_supported(),
    optim ="adamw_torch",
    lr_scheduler_type ="cosine",
    report_to="none",
)